In [2]:
import spacy
import re
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop

In [3]:
#le nom du fichier contenant le dataset
dataset = "dataset_hate" 

#le nom du fichier contenant la stoplist
stoplist = "stoplist.txt"

#le nom du fichier dans lequel le dataset nettoyé sera sauvegardé
clean_dataset = "clean_dataset_hate"

In [4]:
with open(stoplist, "r") as f:
    stopwords = f.read().splitlines()

In [5]:
#importer le vocabulaire français
nlp = spacy.load("fr_core_news_sm")

In [6]:
#Récupération des tweets
with open(dataset, "r") as file:
    raw_data = file.readlines()

In [7]:
%%time 

#transformer les tweets en objet spacy
docs = list(nlp.pipe(raw_data))

CPU times: user 56.2 s, sys: 118 ms, total: 56.3 s
Wall time: 56.3 s


In [8]:
%%time

#tokeniser et lemmatiser l'ensemble des tweets
""" -enlever la ponctuation,
    - enlever les espaces
    -enlever les email
    -enlever les urls"""
first_clean_data = [[token.lemma_.lower() for token in doc 
                               if not token.is_punct and not token.is_space and not token.like_url and not token.like_email and token.tag_!= 'Num'] for doc in docs]

CPU times: user 224 ms, sys: 24 ms, total: 248 ms
Wall time: 246 ms


In [9]:
#Deuxième passe de nettoyage
def clean(msg):
    clean_msg = []
    for word in msg:
        #Supprimer tout ce qui n'est pas des lettres
        clean_word = re.sub(r'\W', '', word)
        #Supprimer les chiffres
        clean_word = re.sub(r'\d', '', clean_word)
        if clean_word not in stopwords : 
            clean_msg.append(clean_word)
    return clean_msg

In [10]:
%%time
clean_data = []

#Nettoyage des tweets
for msg in first_clean_data:
    clean_data.append(clean(msg))

CPU times: user 2.13 s, sys: 3.96 ms, total: 2.13 s
Wall time: 2.13 s


In [11]:
#On a bien uniquement du texte
print(raw_data[0], end='')
print(first_clean_data[0])
print(clean_data[0])

pov tu construis un rideau avec un sac à linge sale et des bd
['pov', 'tu', 'construire', 'un', 'rideau', 'avec', 'un', 'sac', 'à', 'linge', 'sale', 'et', 'de', 'bd']
['pov', 'construire', 'rideau', 'sac', 'linge', 'sale', 'bd']


In [13]:
#enregistrer sur le disque
with open(clean_dataset, "w") as f:
    for msg in clean_data:
        f.write(" ".join(msg) + '\n')